In [ ]:
import sqlite3

# Connect to the SQLite database
connection = sqlite3.connect('unimod.db')  # path to your SQLite file

# Create a cursor object to interact with the database
cursor = connection.cursor()

# Example query: List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

table_name = 'modifications'
# Fetch column names using PRAGMA table_info
cursor.execute(f"PRAGMA table_info({table_name});")
columns = cursor.fetchall()

# Print the column names
print(f"Column names in table '{table_name}':")
for column in columns:
    print(column[1])  # column[1] is the column name in the tuple

# Example query: Fetch data from the table
cursor.execute(f"SELECT * FROM {table_name};")
rows = cursor.fetchall()

# Print the fetched rows
for row in rows:
    print(row)

# Close the connection
connection.close()

In [ ]:
import xmlschema
import pandas as pd

input_dir = 'input'
output_dir = 'output'

schema = xmlschema.XMLSchema(f'{input_dir}/unimod_tables_1.xsd')
is_valid = schema.is_valid(f'{input_dir}/unimod_tables.xml')
print(f"XML is valid: {is_valid}")


data = schema.to_dict(f'{input_dir}/unimod_tables.xml')
data = {key : val for key,val in data.items() if key not in ['@majorVersion', '@minorVersion', '@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', 'xrefs', 'xref_sources']}
data = {key : pd.DataFrame.from_dict(val[f'{key}_row'], orient = 'columns') for key, val in data.items()}

specificity = data['specificity'].rename(columns = {'@record_id' : '@spec_key'})
unimod_modifications = data['modifications'].rename(columns = {'@record_id' : '@mod_key'})
positions = data['positions'].rename(columns = {'@record_id' : '@position_key'})
classifications = data['classifications'].rename(columns = {'@record_id' : '@classifications_key'})
# specnl = data['spec2nl']
# modbrick = data['mod2brick'].groupby('@mod_key')[['@brick', '@num_brick']].agg(list)
# modbrick = modbrick.explode(list(modbrick.columns))

amino_acids = data['amino_acids'].apply(lambda x: {x['@three_letter'] : {'C' : x['@num_C'], 'N' : x['@num_N'], 'O' : x['@num_O'], 
                                          'H' : x['@num_H'], 'S' : x['@num_S'], 'Se' : x['@num_Se']}}, axis = 1)
aa_composition = {}
for entry in amino_acids:
    for key, value in entry.items():
        aa_composition[key] = value
aa_composition['Leu/Ile'] = aa_composition['Leu']
aa_composition['Ile/Leu'] = aa_composition['Leu']

aa_three_to_one_letter = data['amino_acids'][['@three_letter', '@one_letter']]
aa_three_to_one_letter = {three_letter : one_letter for three_letter, one_letter in 
                          zip(aa_three_to_one_letter['@three_letter'], aa_three_to_one_letter['@one_letter'])}
aa_three_to_one_letter['Leu/Ile'] = 'I/L'
aa_three_to_one_letter['Ile/Leu'] = 'I/L'

merged = unimod_modifications.merge(specificity, how = 'left', on = '@mod_key')
merged = merged.merge(positions, how = 'left', on = '@position_key')
merged = merged.merge(classifications, how = 'left', on = '@classifications_key')

aa_subs = merged[merged['@classification'] == 'AA substitution']
aa_subs['Base AA'] = aa_subs['@full_name'].str.split('->').str[0]
aa_subs['Sub AA'] = aa_subs['@full_name'].str.split('->').str[1].str.split(' ').str[0]
aa_subs['Base Composition'] = aa_subs['Base AA'].map(aa_composition)
aa_subs['Sub Composition'] = aa_subs['Sub AA'].map(aa_composition)
aa_subs['Base AA'] = aa_subs['Base AA'].map(aa_three_to_one_letter)
aa_subs['Sub AA'] = aa_subs['Sub AA'].map(aa_three_to_one_letter)
aa_subs = aa_subs.drop_duplicates(subset = ['@position', 'Base AA', 'Sub AA'])

aa_subs = aa_subs[aa_subs['@username_of_poster'] == 'unimod']

isoleucine = aa_subs[aa_subs['Base AA'] == 'I/L']
leucine = aa_subs[aa_subs['Base AA'] == 'I/L']
isoleucine['Base AA'] = 'I'
leucine['Base AA'] = 'L'
aa_subs = pd.concat([aa_subs[aa_subs['Base AA'] != 'I/L'], isoleucine, leucine], axis = 0)

aa_subs.to_csv(f'{output_dir}/unimod_aa_subs.csv', index = False)

ptm = merged[merged['@classification'] == 'Post-translational']
ptm = ptm[(ptm['@username_of_poster'] == 'unimod') | (ptm['@approved'] == 1)]
ptm_1 = ptm[~ptm['@one_letter'].isin(['N-term', 'C-term'])]
ptm_2 = ptm[ptm['@one_letter'].isin(['N-term', 'C-term'])]
all_aa = data['amino_acids']['@one_letter'].values
all_aa = [aa for aa in all_aa if aa not in ['-', 'N-term', 'C-term']]
expanded_ptm_2 = []
for _, row in ptm_2.iterrows():
    for aa in all_aa:
        new_row = row.copy()
        new_row['@one_letter'] = aa
        expanded_ptm_2.append(new_row)
ptm_2 = pd.DataFrame(expanded_ptm_2)
ptm = pd.concat([ptm_1, ptm_2], ignore_index = True)

ptm.to_csv(f'{output_dir}/unimod_ptm.csv', index = False)

In [ ]:
other_categories = merged[merged['@classification'].isin(['Artefact', 'Chemical derivative', 'Multiple', 'Post-translational'])]
other_categories = other_categories[(other_categories['@username_of_poster'] == 'unimod') | (other_categories['@approved'] == 1)]
other_1 = other_categories[~other_categories['@one_letter'].isin(['N-term', 'C-term'])]
other_2 = other_categories[other_categories['@one_letter'].isin(['N-term', 'C-term'])]
all_aa = data['amino_acids']['@one_letter'].values
all_aa = [aa for aa in all_aa if aa not in ['-', 'N-term', 'C-term']]
expanded = []
for _, row in other_2.iterrows():
    for aa in all_aa:
        new_row = row.copy()
        new_row['@one_letter'] = aa
        expanded.append(new_row)
other_2 = pd.DataFrame(expanded)
other_categories = pd.concat([other_1, other_2], ignore_index = True)
other_categories.to_csv(f'{output_dir}/unimod_other_categories.csv', index = False)

In [ ]:
merged.to_csv(f'{output_dir}/unimod_merged_only.csv', index = False)